In [ ]:
!pip install transformers
!pip install torch
!pip install datasets

# Generating CN

In [ ]:
import os
from IPython.display import clear_output
import json
import csv
import pandas
import time
import random
from collections import Counter
from statistics import mean, median, stdev
import numpy as np
import matplotlib.pyplot as plt
import sys
from functools import reduce
from generate import *

df = pandas.read_csv(path_to_hscn)
df = df[df.Assigned_to == 'test'].reset_index(drop=True)

________
# with GPT-2

In [ ]:
model_type = 'GPT-2'
model_path = path_to_model
generated_cns = generate_cn(model_type, model_path, list_hs = df['HS'], decoding = 'top-p')
prepare_output_df(generated_cns, df, model_type, 'top-p')

________
# with DialoGPT


In [ ]:
model_type = 'DialoGPT'
model_path = path_to_model
generated_cns = generate_cn(model_type, model_path, list_hs = df['HS_ed'], decoding = 'top-k')
prepare_output_df(generated_cns, df, model_type, 'top-k')

___
# with BERT


In [ ]:
model_type = 'BERT'
model_path = path_to_model
generated_cns = generate_cn(model_type, model_path, list_hs = df['HS'], decoding = 'beam-search')
prepare_output_df(generated_cns, df, model_type, 'beam-search')

_____
# with BART

In [ ]:
model_type = 'BART'
model_path = path_to_model
generated_cns = generate_cn(model_type, model_path, list_hs = df['HS'], decoding = 'k-p')
prepare_output_df(generated_cns, df, model_type, 'k-p')

___
# with T5

In [ ]:
model_type = 'T5'
model_path = path_to_model
generated_cns = generate_cn(model_type, model_path, list_hs = df['HS'], decoding = 'top-p')
prepare_output_df(generated_cns, df, model_type, 'top-p')

# with all models and decoding mechanisms at once

In [ ]:
model_path = {'T5':path_to_T5,
               'GPT-2':path_to_GPT2,
               'DialoGPT':path_to_DialoGPT,
               'BERT':path_to_BERT,
               'BART':path_to_BART}

out_dfs = []
for model_type in model_path:
  for deco in ['beam-search', 'top-p','top-k', 'k-p']:
    generated_cns = generate_cn(model_type, model_path[model_type], list_hs = df['HS_ed'][:2], decoding = deco)
    out_dfs.append(prepare_output_df(generated_cns, df[:2], model_type, deco))

common_cols = set(out_dfs[0].columns)
for d in out_dfs[1:]:
  common_cols.intersection_update(d.columns)

generations = reduce(lambda  left,right: pd.merge(left,right,on=list(common_cols), how='outer'), out_dfs)